<a href="https://colab.research.google.com/github/luisfelippe/ann/blob/main/pipeline_classificacao_imdb_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q -U datasets
!pip install -q -U transformers

In [2]:
from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('HF_TOKEN'))

In [3]:
from datasets import load_dataset

dataset = load_dataset('stanfordnlp/imdb', trust_remote_code=True)

print(dataset)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [4]:
train_ds = dataset["train"]
test_ds = dataset["test"]
unsup_ds = dataset['unsupervised']

train_val_split = train_ds.train_test_split(test_size=0.2)
train_ds = train_val_split["train"]
val_ds = train_val_split["test"]

print(f'Train: {len(train_ds)}')
print(f'Validation: {len(val_ds)}')
print(f'Test: {len(test_ds)}')

Train: 20000
Validation: 5000
Test: 25000


In [5]:
train_comments = train_ds['text']
train_labels = train_ds['label']

val_comments = val_ds['text']
val_labels = val_ds['label']

test_comments = test_ds['text']
test_labels = test_ds['label']

In [ ]:
train_comments[0]

"This has to be THE WORST film I have ever seen. I bought the DVD and it didn't work, well the DVD worked fine, its just the film didn't, in fact its so bad that I think non of the actors have worked since (or before judging from the acting). There is no real plot to speak of and no real horror. The production values are rubbish even for a low budget film with some outside scenes being hard to hear due to wind on the microphone.<br /><br />All in all it was so bad a film that a viewer could think it was going to turn into a porno if they didn't know any better (which have been an improvement). There are only two lines in the film that are funny, both of which revolve around the transvestite prostitute. (Although they hardly merit watching all 76 minutes for)<br /><br />As for the marketing of the DVD I feel thoroughly cheated. I mean reading the back I expected it to be bad:<br /><br />He chose his weapons. He selected his victims. He picked his nose. He turned into a GIRL!<br /><br />

In [ ]:
train_labels[0]

0

In [6]:
from collections import Counter

print(f'Train Labels Distribution: {Counter(train_labels)}')
print(f'Validation Labels Distribution: {Counter(val_labels)}')
print(f'Test Labels Distribution: {Counter(test_labels)}')

Train Labels Distribution: Counter({0: 10031, 1: 9969})
Validation Labels Distribution: Counter({1: 2531, 0: 2469})
Test Labels Distribution: Counter({0: 12500, 1: 12500})


In [7]:
model_tuple = ('bert_base',
               'google-bert/bert-base-cased')

num_epochs = 10
num_classes = 2

max_input_len = 256
batch_size = 32

optimizer = 'adamw_torch'

In [8]:
from transformers import AutoTokenizer, \
AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_tuple[1])

model = AutoModelForSequenceClassification.from_pretrained(
    model_tuple[1], num_labels=num_classes)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
print(f'Parâmetros: {model.num_parameters():,}')

Parâmetros: 108,311,810


In [9]:
import torch
import torch.nn.functional as f

y_train = torch.tensor(train_labels)
y_val = torch.tensor(val_labels)
y_test = torch.tensor(test_labels)

# Removing one-hot encoding as it's not needed by the Trainer and causes issues with compute_metrics
# y_train = f.one_hot(y_train, num_classes=num_classes)
# y_val = f.one_hot(y_val, num_classes=num_classes)
# y_test = f.one_hot(y_test, num_classes=num_classes)

print(f'{train_labels[0]} --> {y_train[0]}')

1 --> 1


In [ ]:
# from datasets import Dataset

# train_dict = {
#     'text': train_comen,
#     'label': y_train
# }

# val_dict = {
#     'text': validation_tweets,
#     'label': y_val
# }

# test_dict = {
#     'text': test_tweets,
#     'label': y_test
# }

# train_dataset = Dataset.from_dict(train_dict)
# val_dataset = Dataset.from_dict(val_dict)
# test_dataset = Dataset.from_dict(test_dict)

In [ ]:
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})

In [10]:
def tokenize_text(examples, tokenizer_, max_len_):
  return tokenizer_(
      examples['text'],
      max_length=max_len_,
      padding='max_length',
      truncation=True
  )

In [11]:
encoded_train_dataset = train_ds.map(
    lambda batch: tokenize_text(batch, tokenizer, max_input_len),
    batched=True,
    batch_size=batch_size
)

encoded_val_dataset = val_ds.map(
    lambda batch: tokenize_text(batch, tokenizer, max_input_len),
    batched=True,
    batch_size=batch_size
)

encoded_test_dataset = test_ds.map(
    lambda batch: tokenize_text(batch, tokenizer, max_input_len),
    batched=True,
    batch_size=batch_size
)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
encoded_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 20000
})

In [ ]:
print(encoded_train_dataset[0])

{'text': "it aint bad, but it aint good. it is just entertaining.<br /><br />as a comedy which it is supposed to be, it's dreadful. not many laughs at all as every joke in the movie has been done a million times before.<br /><br />it's a shame as all the actors in the film are great usually, but none of them really do much. and the ending sucks.", 'label': 0, 'input_ids': [101, 1122, 9562, 1204, 2213, 117, 1133, 1122, 9562, 1204, 1363, 119, 1122, 1110, 1198, 15021, 119, 133, 9304, 120, 135, 133, 9304, 120, 135, 1112, 170, 3789, 1134, 1122, 1110, 3155, 1106, 1129, 117, 1122, 112, 188, 25671, 119, 1136, 1242, 12375, 1120, 1155, 1112, 1451, 8155, 1107, 1103, 2523, 1144, 1151, 1694, 170, 1550, 1551, 1196, 119, 133, 9304, 120, 135, 133, 9304, 120, 135, 1122, 112, 188, 170, 10499, 1112, 1155, 1103, 5681, 1107, 1103, 1273, 1132, 1632, 1932, 117, 1133, 3839, 1104, 1172, 1541, 1202, 1277, 119, 1105, 1103, 3830, 22797, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [12]:
import numpy as np

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  # Decode one-hot encoded labels to integer labels (no longer needed after removing one-hot encoding)
  # labels = np.argmax(labels, axis=-1)
  preds = np.argmax(logits, axis=-1)
  acc = accuracy_score(labels, preds)
  f1 = f1_score(labels, preds)
  return {
      'acc': acc,
      'f1': f1
  }

In [13]:
from transformers import TrainingArguments, Trainer, \
  EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='bert_base_imdb',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=1,
    greater_is_better=True,
    metric_for_best_model='f1',
    optim=optimizer,
    report_to='none',
    push_to_hub=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Acc,F1
1,0.240800,0.238565,0.909600,0.912301
2,0.162500,0.273396,0.903000,0.900757
3,0.061600,0.390753,0.906400,0.904568


TrainOutput(global_step=1875, training_loss=0.1962443645834923, metrics={'train_runtime': 2780.1324, 'train_samples_per_second': 71.939, 'train_steps_per_second': 2.248, 'total_flos': 7893331660800000.0, 'train_loss': 0.1962443645834923, 'epoch': 3.0})

In [15]:
trainer.save_model('best_model/')
tokenizer.save_pretrained('best_model/')

model.push_to_hub('felippelfcc/bert_base_imdb')
tokenizer.push_to_hub('felippelfcc/bert_base_imdb')

README.md: 0.00B [00:00, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/felippelfcc/bert_base_imdb/commit/257149a5d0ddeb4f9d422e3c6242698a67bcef19', commit_message='Upload tokenizer', commit_description='', oid='257149a5d0ddeb4f9d422e3c6242698a67bcef19', pr_url=None, repo_url=RepoUrl('https://huggingface.co/felippelfcc/bert_base_imdb', endpoint='https://huggingface.co', repo_type='model', repo_id='felippelfcc/bert_base_imdb'), pr_revision=None, pr_num=None)

In [16]:
predictions = trainer.predict(encoded_test_dataset)

In [21]:
# calcular a metrica dos predictions
evaluation_metrics = compute_metrics((predictions.predictions, predictions.label_ids))
print(evaluation_metrics)

{'acc': 0.9074, 'f1': 0.9100097181729835}
